In [1]:
import os
sep = os.sep
import numpy as np

In [2]:
file = '02_test.tif'

In [17]:
CONF = {
    'Dirs': {
        'image': 'data' + sep + 'DRIVE' + sep + 'images',
        'mask': 'data' + sep + 'DRIVE' + sep + 'mask',
        'truth': 'data' + sep + 'DRIVE' + sep + 'manual',
        'logs': 'LOGS_2019' + sep + 'DRIVE',
        'splits_json': 'data' + sep + 'DRIVE' + sep + 'splits'
    },

    'Funcs': {
        'truth_getter': lambda file_name: file_name.split('_')[0] + '_manual1.gif',
        'mask_getter': lambda file_name: file_name.split('_')[0] + '_mask.gif',
    }
}

In [18]:
from PIL import Image as IMG
from utils import img_utils as iu
from utils.img_utils import Image
from sklearn.metrics import f1_score
from scipy.ndimage.morphology import distance_transform_edt

In [95]:
for file in os.listdir(CONF['Dirs']['image']):
    
    if 'training' in file:
        continue
    file = file.split('.')[0]
    
    image = Image()
    image.load_file(CONF['Dirs']['image'], file+'.tif')
    image.load_ground_truth(CONF['Dirs']['truth'], CONF['Funcs']['truth_getter'])
    image.load_mask(CONF['Dirs']['mask'], CONF['Funcs']['mask_getter'])
    
    pred_file =  file + '.png'
    image.u_1_1 = np.array(IMG.open(CONF['Dirs']['logs'] + sep + 'UNET_1_1' + sep + 'pred_'+pred_file))
    image.u_1_100 = np.array(IMG.open(CONF['Dirs']['logs'] + sep + 'MAPNET_1_100_1' + sep + pred_file))
    
    image.u_1_1[image.u_1_1==255] = 1
    image.u_1_100[image.u_1_100==255] = 1
    image.ground_truth[image.ground_truth==255] = 1 
    
    f1 = f1_score(image.ground_truth.flatten(), image.u_1_100.flatten())
    
    gt = image.ground_truth.copy()
    gt[gt==255] = 1
    edt_gt = distance_transform_edt(gt)
    
    edt = edt_gt.copy()
    edt[edt==0] = 1
    edt = 1/edt
    
    f1_miniunet = f1_score(gt.flatten(), image.u_1_100.flatten(),\
         sample_weight=edt.flatten())
    f1_plain = f1_score(gt.flatten(), image.u_1_1.flatten(),\
         sample_weight=edt.flatten())
    print(file,  round(f1, 5), round(f1_plain, 5), round(f1_miniunet, 5), round(f1_miniunet-f1_plain, 5))

01_test 0.83223 0.78619 0.78915 0.00296
02_test 0.85543 0.80007 0.80344 0.00337
03_test 0.81951 0.74049 0.75269 0.01219
04_test 0.8334 0.76511 0.76979 0.00468
05_test 0.81334 0.75002 0.75072 0.0007
06_test 0.80849 0.73936 0.74357 0.00421
07_test 0.82161 0.75154 0.76246 0.01092
08_test 0.80135 0.73719 0.74044 0.00325
09_test 0.81739 0.75994 0.7646 0.00466
10_test 0.81523 0.75605 0.75931 0.00325
11_test 0.80988 0.75477 0.75827 0.0035
12_test 0.83383 0.78359 0.78354 -5e-05
13_test 0.83016 0.76443 0.77428 0.00985
14_test 0.83068 0.77496 0.77936 0.0044
15_test 0.82056 0.76471 0.77491 0.01019
16_test 0.83799 0.79869 0.79756 -0.00113
17_test 0.80867 0.76014 0.75994 -0.00021
18_test 0.81195 0.76938 0.76737 -0.00202
19_test 0.85709 0.82393 0.8237 -0.00023
20_test 0.81161 0.77711 0.76916 -0.00795


In [88]:
IMG.fromarray(edt/np.max(edt) * 255, dtype=np.uint8)

TypeError: fromarray() got an unexpected keyword argument 'dtype'

In [28]:
for i in range(10):
    rngea = np.arange(5, 50, 5)
    rngeb = np.arange(200, 250, 5)
    print(random.choice(rngea), random.choice(rngeb))

5 230
40 225
45 225
35 225
5 220
45 200
5 235
45 240
20 240
40 235


In [1]:
from sklearn.mixture import GaussianMixture 

In [8]:
import random